# Setup

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression

sns.set(style="darkgrid")

# Import the dataset
The dataset represents the stocks with market capitalization higher than 3000 mil USD. The dataset was retrieved using the stock screener of ZACKS.com

In [ ]:
df = pd.read_csv('./zacks_stocks_custom_screen_2024-08-21.csv')
df.head()

Let’s just rename those columns so we can more easily refer to them later.

In [ ]:
# EPS0 = Earning Per Share in year 0 (last year)
# EPS1 = Earning Per Share in year 1 (current year)
# EPS2 = Earning Per Share in year 2 (next year)
df = df.rename(columns={'Last Close':'Price', 'Last Yr`s EPS (F0) Before NRI':'EPS0', 'F1 Consensus Est.':'EPS1', 'F2 Consensus Est.':'EPS2'})
df.head()

Let's rearrange the columns in way that on we can interpret them better. We move columns 'Market Cap (mil)', 'Price', 'EPS0', 'EPS1' and 'EPS2' are shown next to each other.

In [ ]:
df = df[['Company Name', 'Ticker', 'Month of Fiscal Yr End', 'Last Reported Fiscal Yr  (yyyymm)', 'Sector', 'Industry', 'Exchange', 'Market Cap (mil)', 'Price', 'EPS0', 'EPS1', 'EPS2']]
df.head()

# Explore the dataset

Let's have a look at the dataset and undestand its characteristics.

In [ ]:
# output the length of the dataset
len(df)

In [ ]:
# output statistical description of the dataset
df.describe()

In [ ]:
df.dtypes

Let's check the proportion of missing data

In [ ]:
df.isnull().sum()

# Clean the dataset

We see that 66 company names are missing but since no ticker is missing this doesn't bother us as the ticker is a unique identifier for each stock. 

We also see that 97 entries for EPS0 (Earning per Share in the year 0) are missing this is roughly 5% of our data set. We also see that rougly 200 entries for both EPS1 and EPS2 are missing. These respresent each roughly 10% of our dataset. If we assume that there is no intersection of the missing values for EPS0, EPS1 and EPS2 the missing values would accout tofether for roughly 25% of our data set.

Let's clean the dataset from these missing values and check the real effect on the dataframe and decide if we can live with that or we will need to find more sophisticated ways to handle the missing values.

In [ ]:
df = df[df['EPS0'].notna() & df['EPS1'].notna() & df['EPS2'].notna()]
len(df)

We see that the cleaned dataset contains 1802 stocks which is 10% less than the original dataset with 2028 stocks. For the purpose of our data analysis this subset carries more than enough evidence to identify which industries are growing/contracting most and to identify potential stocks outliers which are especially positive/negative. Therefore, we will continue with the obtained dataset without further investigation of the missing values.

A final check shows us that our dataset is clean of missing values exept for one company name value but we don't care about that since we have the Ticker instead.

In [ ]:
df.isnull().sum()

Great! We have finished our data set setup.

# Business Goal 1
# Identify Sectors & Industries with highest Earning Growth/Contraction

## 1.1 Rank Sectors according to Earning Growth

Our first busieness goal in this section is to understand which Sectors are expected to have the strongest Grwoth/Contraction in current and next year according to the Forward Looking Earning Growth Expectation for year 1 and year 2 (EG1 and EG2).

## 1.1.1 Prepare the dataset

First, we will add some columns to our dataframe which represents useful financial metrics and prepare the dataset for ranking according to earning growth.

The colums which we will all are derived by simple addition/division of the stock price, EPS0, EPS1 and EPS2 values we already have in our dataframe. First, We add two columns "EG1" i.e. earning grwoth in year 1 and "EG2" this is earning growth in year 2.

In [ ]:
def earning_growth(eps0, eps1):
    '''
    INPUT - eps0 - flaot - with the earning per share for year 0
            eps1 - flaot - with the earning per share for year 1
    OUTPUT - 
            eg - float - earning per share growth from year 0 to year 1
    '''
    if eps0<0:
        if eps1 >= 0:
            return 1.0
    if eps0==0:
        if eps1>0:
            return 1.0
        if eps1<0:
            return -1.0
        if eps==0:
            return 0
        
    eg=(eps1-eps0)/abs(eps0)
    
    return eg

df['EG1']= df.apply(lambda row: earning_growth(row['EPS0'], row['EPS1']), axis=1)

df['EG2']= df.apply(lambda row: earning_growth(row['EPS1'], row['EPS2']), axis=1)


Now, let's explore the struture of our dataframe:

In [ ]:
df.head()

We can see that the columns have been successfully added.

We will also add the columns below which we will need later in our analysis:
    - "PE1" and "PE2" i.e. price to earning ratios for current and next year respectively. 
    - "PEG1" and "PEG2" i.e. price to earning growth ratios for current and next year respectively.

In [ ]:
PE1 = (df['Price']/df['EPS1'])
df['PE1']=PE1

PE2 = (df['Price']/df['EPS2'])
df['PE2']=PE2

In [ ]:
PEG1 = (df['PE1']/(df['EG1'])*100)
df['PEG1']=PEG1

PEG2 = (df['PE2']/(df['EG2'])*100)
df['PEG2']=PEG2

Let's see the newly added columns.

In [ ]:
df.head()

In [ ]:
df.describe()

In the description table above we can see that the EG1 and EG2 columns have no division by zero or any missing values.

We can see also see that we have EG1 and EG2 max values of respectively 16.2 and 287.0. Since these values are to be interpreted as percentage values they mean a maximum earning growth of 1620% in year 1 and of 28700% in year 2. These values are extremly high and unrealistic. Probably, they result from a division by a very small number (Law of Small Numbers) or from a wrong entry.

Similarly, we see that we have EG1 and EG2 min values of -23.0 and -4.72 or -2300% and -472%

For the purpose of our analysis, we will consider only growth rate between -400% and +400% or between -4.0 and +4.0 to be realistic. All other values will be considered as outliers to be deleted from the main dataset "df" to be added to the dataframe "outliers" for further investigation in later analysis stages.

In [ ]:
#save the stocks with EG1>threshold or EG1<-threshold for later analysis
threshold = 4
outliers = df[df['EG1']>threshold]
outliers = outliers.append(df[df['EG1']<-threshold])

outliers

In [ ]:
# drop the stockswith EG1>threshold or EG1<-threshold from the dataset
df = df[~(df['EG1']>threshold)]
df = df[~(df['EG1']<-threshold)]

In [ ]:
#save the stocks with EG2>threshold or EG2<-threshold for later analysis
outliers = outliers.append(df[df['EG2']>threshold])
outliers = outliers.append(df[df['EG2']<-threshold])

outliers

### 1.1.2 Rank Sectors according to Forward Looking Earning Growth Expectation for current year (EG1)

Now, we are done witht the preparation of the dataset and we will start the ranking according to earning growth 1.

In [ ]:
# drop the stocks with EG2>threshold or EG2<-threshold from the dataset
df = df[~(df['EG2']>threshold)]
df = df[~(df['EG2']<-threshold)]

Now we will group the data set by Sector and calculate the average EG1 for each sector as well as calculate the 50% and 75% quantiles for each sector to undestand the sensitivity of the calculared averages.

In [ ]:
group_by_sector = df.loc[:, ['Sector', 'EG1', 'EG2']].groupby('Sector')

avgs = group_by_sector.EG1.mean()
avgs = avgs.rename('mean EG1')
qtls50 = group_by_sector.EG1.quantile(0.5)
qtls50 = qtls50.rename('50% EG1')
qtls75 = group_by_sector.EG1.quantile(0.75)
qtls75 = qtls75.rename('75% EG1')

Now that we have done all the calculation of each sector let's plot them.

In [ ]:
# plot the mean earning growth mean for year 1 across sectors
avgs.plot(kind='barh', legend="Mean")
plt.title("Mean EG1 across Sectors")
plt.show()

# plot the 50% earning growth for year 1 across sectors 

qtls50.plot(kind='barh', legend="75Q")
plt.title("50%P EG1 across Sectors")
plt.show()

# plot the 75% earning growth for year 1 across sectors 

qtls75.plot(kind='barh', legend="75Q")
plt.title("75%P EG1 across Sectors")
plt.show()

The plot above for Sector Rankings accroding to EG1 allows us to get a first glimpse of which industries are expecting strongest Growth/Contraction in current year.

Though the plot gives us a first impression of where strongest Grwoth/Contraction is to be expected in the current year, we aknowledge that there are big differences between the mean, 50% quantile and 75% quantile plots. This indicates that the EG1 distribution within most sectors is realtively unhomogeneous.

So, let's see if a ranking according to according to Forward Looking Earning Growth Expectation for next year (EG2) will give a different picture.

### 1.1.3 Rank Sectors according to Forward Looking Earning Growth Expectation for next year (EG2)

In [ ]:
avgs = group_by_sector.EG2.mean()
avgs = avgs.rename('mean EG2')
qtls50 = group_by_sector.EG2.quantile(0.5)
qtls50 = qtls50.rename('50% EG2')
qtls75 = group_by_sector.EG2.quantile(0.75)
qtls75 = qtls75.rename('75% EG2')

# plot the mean earning growth mean for year 2 across sectors
avgs.plot(kind='barh', legend="Mean")
plt.title("Mean EG2 across Sectors")
plt.show()

# plot the 50% earning growth for year 2 across sectors

qtls50.plot(kind='barh', legend="75Q")
plt.title("50%P EG2 across Sectors")
plt.show()

# plot the 75% earning growth for year 2 across sectors

qtls75.plot(kind='barh', legend="75Q")
plt.title("75%P EG2 across Sectors")
plt.show()

The plot above for Sector Rankings accroding to EG2 gives us an second impression about which sectors are expected to grow/contract most in year 2 (next year).

The differences between the mean, 50% quantile and 75% quantile plots for EG2 are smaller than those for EG1. This indicates that the EG2 distribution within most sectors is more homogeneously distributed than the EG1 distribution. If we take the plots of EG1 and EG2 together we can get a first feeling about which sectors to grow/contract in the curent and next years.

Nonetheless, we suspect that these plots of EG1 and EG2 can -at best- give us a rough directional indication of where earning growth/contraction to be expected. So, Let's look at a statistical description of the earning growth data.

In [ ]:
group_by_sector.describe()

Looking at the table above, the unhomogenity of earning growth rate distribution is confirmed by looking at the standard deviation of the EG1 and EG2 means as well as the min and max values.

Therefore, we will proceed to drilling down deeper by ranking the industries according the earning growth data. By doing this we expect the earning growth distribution within indutries to be more homegenous than within sectors. 

## 1.2 Rank Industries according to Earning Growth

Now we will group the data set by Industry and calculate the average EG1 and EG2 for each Industry.

In [ ]:
group_by_industry = df.loc[:, ['Industry', 'EG1', 'EG2']].groupby('Industry')

Let's describe the sorted data:

In [ ]:
group_by_industry.describe()

From the table above we see that we have more than 200 industries in our dataset. 

Since we are only interested in knowing the industries with highest growth/contraction, in the follwing we will plot the Top-10 industries with highest earning growth and the Bottom-10 industries with lowest earning growth.

### 1.2.2 Rank Top-10/Bottom-10 Industries according to Forward Looking Earning Growth Expectation for current year (EG1)

In [ ]:
# compute average earning grwoth for year 1 for each Industry to identify which indutries had the highest/lowest growth rates
avgs = group_by_industry.EG1.mean()
avgs = avgs.rename('mean EG1')

#sort industries from highest to lowest EG1
sorted_avgs = avgs.sort_values(ascending=False)

top_bottom_eg1 = sorted_avgs.head(10)
top_bottom_eg1 = top_bottom_eg1.append(sorted_avgs.tail(10))

# plot the 10 industries with highest/lowest average earning growth for year 1
top_bottom_eg1.plot(kind='barh', legend="Mean EG1")
plt.title("Top/Flop 10 Industries with highest/lowest average EG1")
plt.show()

sorted_avgs



### 1.2.2 Rank Top-10/Bottom-10 Industries according to Forward Looking Earning Growth Expectation for next year (EG2)

In [ ]:
# compute average earning grwoth for year 2 for each Industry to identify which indutries had the highest/lowest growth rates
avgs = group_by_industry.EG2.mean()
avgs = avgs.rename('mean EG2')

#sort industries from highest to lowest EG2
sorted_avgs = avgs.sort_values(ascending=False)

top_bottom_eg2 = sorted_avgs.head(10)
top_bottom_eg2 = top_bottom_eg2.append(sorted_avgs.tail(10))

# plot the 10 industries with highest/lowest average earning growth for year 2
top_bottom_eg2.plot(kind='barh', legend="Mean EG2")
plt.title("Top/Bottom 10 Industries with highest/lowest average EG2")
plt.show()

sorted_avgs

In [ ]:
df[df["Industry"]=="Agriculture - Products"]

# Business Goal 2
# Identify Potential Trade Ideas (Long/Short)

## 2.1 Preapre the dataset

Let's see the description of our data with focus on PE1 and PE2.

In [ ]:
df.describe()

We can see that the PE1 column has some divisions by zero which we will try to understand in the next step.

In [ ]:
#the PE1 columns is obtained by deviding price column by the ep1 columns
#the condition below helps us understand which stocks have 0 USD earning
outliers = outliers.append(df[df['EPS1']==0])
outliers

We can see that only two stocks have the problem of the division by zero. We note that both stocks are "internet" stocks which seem to have bad earnings per shares in period 0 and 1 but which are expected to have better earning per share in period 2. These could be stocks with "revenue growth story". But for now, we cannot be sure about that. We save these stocks to a new dataframe called outliers for further qualitative/quantitative analysis later. And we remove them from the dataframe so that we have a clean dataset to work with in the follwing steps.

In [ ]:
#save the stocks with infinite PE1 or PE2 ratio for later analysis
outliers = outliers.append(df[df['EPS2']==0])
outliers

In [ ]:
# drop the stocks with infinite PE1 or PE2 ratio from the dataset
df = df[~(df['EPS1']==0)]
df = df[~(df['EPS2']==0)]

df.describe()

## 2.2 Identify Potential Long Trade Idea for the Current Year

After finishing the dataset preparation, all what we need to do is to select the industries with the highest and lowest growth from the main dataframe. 

For each industry with top growth we will sort the stocks within that industry from highest to lowest PE ratio. We will then export these stocks of each industry to excel on a seperate tab/sheet for further analysis. 

The trading/financial logic behind this approach is that the stocks with the highest PE ratios within one of the top  growth industries are potential Long Trade Ideas for two main reasons. First, the identified industry has higher grwoth than the average market growth which means that a random stock within that indutry is likely to make higher profits than the marekt average. Second, since the PE ratio is indicative for how much money investors are willing to pay for the earning of a given comppany, by selecting the highest PE ratio we identify the most loved stocks by the market within an already confirmed higher-than average growth industry.

This is done in the follwoing:

In [ ]:
# Create a Pandas Excel writer using openpyxl as the engine.
writer = pd.ExcelWriter("long_short_outliers_year1.xlsx", engine="openpyxl")

# Write each dataframe to a different worksheet.
df.to_excel(writer, sheet_name="Dataset")

#sort outlier from highest to lowest PE2
outliers = outliers.sort_values(by=['PE1'], ascending=False)
outliers.to_excel(writer, sheet_name="Outliers")

In [ ]:
# add the PE1 sorted long trade ideas
for x in range(5):
    #select the industry with highest growth EG1
    industry_df = df[df['Industry']==top_bottom_eg1.index[x]]
    #sort the industry form highest to lowest PE1
    industry_df = industry_df.sort_values(by=['PE1'], ascending=False)
    #output industry dataframe to a seperate excel tab
    sheet_name = top_bottom_eg1.index[x]
    if (len(sheet_name) > 31):
        sheet_name = sheet_name[0:31]
    industry_df.to_excel(writer, sheet_name=sheet_name)

Analogous to the logic explained above, stocks with lowest PE within lowest growth/ highest contraction industry are potential Short Trade Ideas.

In [ ]:
# add the PE1 sorted short trade ideas
for x in range(5):
    #select the industry with lowest growth EG1
    industry_df = df[df['Industry']==top_bottom_eg1.index[len(top_bottom_eg1)-1-x]]
    #sort the industry form lowest to highest PE1
    industry_df = industry_df.sort_values(by=['PE1'], ascending=True)
    #output industry dataframe to a seperate excel tab
    sheet_name = top_bottom_eg1.index[len(top_bottom_eg1)-1-x]
    if (len(sheet_name) > 31):
        sheet_name = sheet_name[0:31]
    industry_df.to_excel(writer, sheet_name=sheet_name)

# Close the Pandas Excel writer and output the Excel file.
writer.close()

## 2.3 Identify Potential Long/Short Trade Ideas for the Next Year


Analogous to the approach above, we generate an excel file with potential Long/Short Trade Ideas for the Next Year (year 2). The only difference is that instead of EG1 and PE1 as selction and sorting criteria here we use EG2 and EP2 for year 2.

In [ ]:
# Create a Pandas Excel writer using openpyxl as the engine.
writer= pd.ExcelWriter("long_short_outliers_year2.xlsx", engine="openpyxl")

# Write each dataframe to a different worksheet.
df.to_excel(writer, sheet_name="Dataset")

#sort outlier from highest to lowest PE2
outliers = outliers.sort_values(by=['PE2'], ascending=False)
outliers.to_excel(writer, sheet_name="Outliers")

In [ ]:
# add the PE2 sorted long trade ideas
for x in range(5):
    #select the industry with highest growth EG2
    industry_df = df[df['Industry']==top_bottom_eg2.index[x]]
    #sort the industry form highest to lowest PE2
    industry_df = industry_df.sort_values(by=['PE2'], ascending=False)
    #output industry dataframe to a seperate excel tab
    sheet_name = top_bottom_eg2.index[x]
    if (len(sheet_name) > 31):
        sheet_name = sheet_name[0:31]
    industry_df.to_excel(writer, sheet_name=sheet_name)

In [ ]:
# add the PE2 sorted short trade ideas
for x in range(5):
    #select the industry with lowest growth EG2
    industry_df = df[df['Industry']==top_bottom_eg2.index[len(top_bottom_eg2)-1-x]]
    #sort the industry form lowest to highest PE2
    industry_df = industry_df.sort_values(by=['PE2'], ascending=True)
    #output industry dataframe to a seperate excel tab
    sheet_name = top_bottom_eg2.index[len(top_bottom_eg2)-1-x]
    if (len(sheet_name) > 31):
        sheet_name = sheet_name[0:31]
    industry_df.to_excel(writer, sheet_name=sheet_name)

# Close the Pandas Excel writer and output the Excel file.
writer.close()

# Business Goal 3
# Predict Stock Price 

In this section, we will try to predict the stock price based on the numerical variables we have. We will base the price prediction on the parameters "Market Cap (mil)", "EPS0", "EPS1" and "EPS2" since these parameters represent the genuine data as imported. All other parmeters like EG1, EG2, EP1 etc. are derived from these.

We use a linear regression for this purpose. We don't expect great prediction precision as it is clear that predicting stock market prices with a linear model is not the smartest thing to do. Nonetheless for the purpose of this project we give it a try and see if we will get the secret to predict stock market prices :)

In [ ]:
# We only consider numerical variables
X = df[["Market Cap (mil)", "EPS0", "EPS1", "EPS2"]]
y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state=42)

#Four steps:

#Instantiate
lm_model = LinearRegression(normalize=True) 

#Fit
lm_model.fit(X_train, y_train)

#Predict
y_test_preds = lm_model.predict(X_test) # Predictions here using X_test and lm_model

#Score
r2_test = r2_score(y_test, y_test_preds)  # Rsquared here for comparing test and preds from lm_2_model

# Print r2 to see result
r2_test

As expected we have an r2_test of 0.66 which is really really bad! We can try to optimize the linear regression modell by adding some categorical variables or changing the test size or the random state. But it's abious that this is not going to work! We cannot predict a highly non-linear behavior like a stock price using a linear regression.

So, we will finish the project here and hope that in the next projects we will have better tools to predict stock price :)